In [1]:
%matplotlib inline
import pandas as pd
import numpy as np
import lightgbm as lgb

from sklearn.model_selection import cross_val_score, TimeSeriesSplit, train_test_split
from sklearn.metrics import mean_squared_error
from lightgbm import LGBMRegressor

import matplotlib.pyplot as plt

DATA = 'data/'

In [62]:
train_df = pd.read_hdf(DATA + 'features_v1.h5', key='features')
test_df =  pd.read_hdf(DATA + 'features_test_v1.h5', key='features')

test_df.set_index('ID', inplace=True)

In [63]:
train_df.head()
test_df.head()

,date_block_num,item_price,item_cnt_day,shop_id,item_id,item_category_id,city_id,item_lag_1,item_lag_3,item_lag_6,...,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_avg,item_category_city_avg,month,days
61200,12,6999.0,0.0,5,3839,24,5,0.000000,0.000000,0.0,...,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1,31
61201,12,899.0,0.0,5,6415,23,5,0.000000,0.032258,0.0,...,0.0,1.838710,1.483871,0.258065,0.322581,0.0,0.083333,0.153153,1,31
61202,12,2090.0,0.0,5,5664,3,5,0.032258,0.000000,0.0,...,0.0,1.000000,0.000000,0.000000,0.000000,0.0,0.083333,0.107143,1,31
61203,12,2599.0,6.0,5,3734,23,5,0.516129,0.645161,0.0,...,0.0,1.838710,1.483871,0.258065,0.322581,0.0,5.583333,0.153067,1,31
61204,12,599.0,0.0,5,6710,30,5,0.000000,0.000000,0.0,...,0.0,4.806452,3.129032,2.741935,1.838710,0.0,0.000000,0.643995,1,31


,shop_id,item_id,item_category_id,city_id,date_block_num,item_lag_1,item_lag_3,item_lag_6,item_lag_9,item_lag_12,...,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_avg,item_category_city_avg,month,days
ID,,,,,,,,,,,,,,,,,,,,,
0,5,5037,19,5,34,0.000000,0.100000,0.033333,0.0,0.033333,...,0.033333,2.500000,4.466667,2.333333,1.900000,1.766667,0.328571,0.278230,11,30
1,5,5320,55,5,34,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,3.466667,4.533333,3.233333,3.433333,3.466667,0.000000,0.130229,11,30
2,5,5233,19,5,34,0.033333,0.033333,0.100000,0.0,0.000000,...,0.000000,2.500000,4.466667,2.333333,1.900000,1.766667,0.000000,0.278230,11,30
3,5,5232,23,5,34,0.000000,0.033333,0.000000,0.0,0.000000,...,0.000000,1.800000,2.433333,1.433333,1.066667,2.000000,0.000000,0.043297,11,30
4,5,5268,20,5,34,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,2.966667,3.200000,2.066667,1.633333,3.166667,0.109524,0.704610,11,30


In [64]:
# test_df.columns

In [65]:
columns = [
    'item_price', 'shop_id', 'item_id', 'item_category_id', 'city_id', 'month', 'days',
    'date_block_num', 'item_lag_1', 'item_lag_3', 'item_lag_6', 'item_lag_9',
    'item_lag_12', 'item_category_lag_1', 'item_category_lag_3',
    'item_category_lag_6', 'item_category_lag_9', 'item_category_lag_12',
    'item_avg', 'item_category_avg', 'item_city_lag_1', 'item_city_lag_3',
    'item_city_lag_6', 'item_city_lag_9', 'item_city_lag_12',
    'item_category_city_lag_1', 'item_category_city_lag_3',
    'item_category_city_lag_6', 'item_category_city_lag_9',
    'item_category_city_lag_12', 'item_city_avg', 'item_category_city_avg'
]

category_columns = list(range(7))

In [66]:
# X = train_df[train_df.date_block_num > 23][columns].values
# y = train_df[train_df.date_block_num > 23].item_cnt_day.values

X = train_df[columns].values
y = train_df.item_cnt_day.values

In [67]:
X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=False, test_size=0.2)

In [68]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'rmse',
    'verbose': 10,
    'num_iterations': 500
}

lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=category_columns)

lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=category_columns)

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_eval,
    early_stopping_rounds=30,
    verbose_eval=10,
    categorical_feature=category_columns)

C:\Users\kipil\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\kipil\Anaconda3\lib\site-packages\lightgbm\basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 30 rounds.
[10]	valid_0's rmse: 1.22754
[20]	valid_0's rmse: 1.14935
[30]	valid_0's rmse: 1.11478
[40]	valid_0's rmse: 1.0932
[50]	valid_0's rmse: 1.07992
[60]	valid_0's rmse: 1.07008
[70]	valid_0's rmse: 1.06248
[80]	valid_0's rmse: 1.05807
[90]	valid_0's rmse: 1.05525
[100]	valid_0's rmse: 1.05351
[110]	valid_0's rmse: 1.05089
[120]	valid_0's rmse: 1.04966
[130]	valid_0's rmse: 1.0464
[140]	valid_0's rmse: 1.04348
[150]	valid_0's rmse: 1.0419
[160]	valid_0's rmse: 1.04021
[170]	valid_0's rmse: 1.03947
[180]	valid_0's rmse: 1.03732
[190]	valid_0's rmse: 1.0361
[200]	valid_0's rmse: 1.03309
[210]	valid_0's rmse: 1.03199
[220]	valid_0's rmse: 1.02903
[230]	valid_0's rmse: 1.0273
[240]	valid_0's rmse: 1.02774
[250]	valid_0's rmse: 1.02637
[260]	valid_0's rmse: 1.0248
[270]	valid_0's rmse: 1.02344
[280]	valid_0's rmse: 1.0221
[290]	valid_0's rmse: 1.02153
[300]	valid_0's rmse: 1.0205
[310]	valid_0's rmse: 1.01843
[320]	valid_0's rmse: 1.0

In [69]:
X_test = test_df[columns].values

In [70]:
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [71]:
submissions_df = pd.read_csv(DATA + "sample_submission.csv.gz")

In [72]:
submissions_df.item_cnt_month = y_pred
submissions_df.to_csv('submissions/lagged_features_submission_.csv', index=False)

In [43]:
submissions_df = pd.read_csv("submissions/lagged_features_submission_.csv")

In [44]:
submissions_df.set_index('ID', inplace=True)

In [45]:
submissions_df.head()

,item_cnt_month
ID,
0,0.553384
1,0.024418
2,1.555526
3,0.819202
4,0.065095


In [46]:
test_df.head()

,shop_id,item_id,item_category_id,city_id,date_block_num,item_lag_1,item_lag_3,item_lag_6,item_lag_9,item_lag_12,...,item_city_lag_12,item_category_city_lag_1,item_category_city_lag_3,item_category_city_lag_6,item_category_city_lag_9,item_category_city_lag_12,item_city_avg,item_category_city_avg,month,days
ID,,,,,,,,,,,,,,,,,,,,,
0,5,5037,19,5,34,0.000000,0.100000,0.033333,0.0,0.033333,...,0.033333,2.500000,4.466667,2.333333,1.900000,1.766667,0.328571,0.278230,11,30
1,5,5320,55,5,34,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,3.466667,4.533333,3.233333,3.433333,3.466667,0.000000,0.130229,11,30
2,5,5233,19,5,34,0.033333,0.033333,0.100000,0.0,0.000000,...,0.000000,2.500000,4.466667,2.333333,1.900000,1.766667,0.000000,0.278230,11,30
3,5,5232,23,5,34,0.000000,0.033333,0.000000,0.0,0.000000,...,0.000000,1.800000,2.433333,1.433333,1.066667,2.000000,0.000000,0.043297,11,30
4,5,5268,20,5,34,0.000000,0.000000,0.000000,0.0,0.000000,...,0.000000,2.966667,3.200000,2.066667,1.633333,3.166667,0.109524,0.704610,11,30


In [47]:
train_df.date_block_num.max() - 12

21

In [48]:
grouped = train_df[train_df.date_block_num > 21].groupby('item_id')

In [49]:
sums = grouped.item_cnt_day.sum()

In [50]:
zeros = sums[sums == 0.]

In [51]:
zeros.shape

(471,)

In [52]:
tmp = test_df.merge(submissions_df, how='left', left_index=True, right_index=True)

In [53]:
tmp.columns

Index(['shop_id', 'item_id', 'item_category_id', 'city_id', 'date_block_num',
       'item_lag_1', 'item_lag_3', 'item_lag_6', 'item_lag_9', 'item_lag_12',
       'item_category_lag_1', 'item_category_lag_3', 'item_category_lag_6',
       'item_category_lag_9', 'item_category_lag_12', 'item_avg',
       'item_category_avg', 'item_price', 'item_city_lag_1', 'item_city_lag_3',
       'item_city_lag_6', 'item_city_lag_9', 'item_city_lag_12',
       'item_category_city_lag_1', 'item_category_city_lag_3',
       'item_category_city_lag_6', 'item_category_city_lag_9',
       'item_category_city_lag_12', 'item_city_avg', 'item_category_city_avg',
       'month', 'days', 'item_cnt_month'],
      dtype='object')

In [54]:
tmp.drop(['shop_id', 'city_id', 'date_block_num',
       'item_lag_1', 'item_lag_3', 'item_lag_6', 'item_lag_9', 'item_lag_12',
       'item_category_lag_1', 'item_category_lag_3', 'item_category_lag_6',
       'item_category_lag_9', 'item_category_lag_12', 'item_avg',
       'item_category_avg', 'item_price', 'item_city_lag_1', 'item_city_lag_3',
       'item_city_lag_6', 'item_city_lag_9', 'item_city_lag_12',
       'item_category_city_lag_1', 'item_category_city_lag_3',
       'item_category_city_lag_6', 'item_category_city_lag_9',
       'item_category_city_lag_12', 'item_city_avg', 'item_category_city_avg',
       'month', 'days'], inplace=True, axis=1)

In [55]:
tmp = tmp.merge(sums.to_frame(), how='left', left_on='item_id', right_index=True)

In [56]:
tmp[(tmp.item_cnt_day == 0) & (tmp.item_cnt_month != 0)].shape

tmp[(tmp.item_cnt_day == 0) & (tmp.item_cnt_month != 0)].head(10)

(19782, 4)

,item_id,item_category_id,item_cnt_month,item_cnt_day
ID,,,,
1,5320,55,0.024418,0.0
4,5268,20,0.065095,0.0
45,5826,65,0.062001,0.0
64,3538,30,0.095500,0.0
65,3571,55,0.163313,0.0
70,3604,55,0.057131,0.0
74,3407,28,0.062822,0.0
75,3408,20,0.087109,0.0
77,3405,30,0.058004,0.0


In [57]:
tmp.loc[tmp.item_cnt_day == 0, 'item_cnt_month'] = 0

In [58]:
submissions_df.item_cnt_month = tmp.item_cnt_month

In [59]:
submissions_df.head()

,item_cnt_month
ID,
0,0.553384
1,0.000000
2,1.555526
3,0.819202
4,0.000000


In [60]:
# submissions_df = pd.read_csv("submissions/lagged_features_submission_2.csv")

In [61]:
submissions_df.to_csv('submissions/lagged_features_submission_.csv')

In [18]:
tmp = test_df.copy()
tmp['predictions'] = y_pred

In [24]:
# tmp[tmp.item_price == 0].predictions.value_counts()

# tmp.loc[tmp.item_price == 0, 'predictions'] = 0

In [25]:
submissions_df.item_cnt_month = tmp.predictions
submissions_df.to_csv('submissions/lagged_features_adjusted_submission.csv', index=False)

In [31]:
train_df.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id,item_lag_1,item_lag_2,item_lag_3,...,item_cumsum,item_cumcount,item_category_cumsum,item_category_cumcount,item_returned_cumsum,item_returned_cumcount,item_category_returned_cumsum,item_category_returned_cumcount,item_cumavg,item_category_cumavg
0,02.01.2013,0,59,22154,999.00,1.0,37,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
1,03.01.2013,0,25,2552,899.00,1.0,58,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0
2,05.01.2013,0,25,2552,899.00,-1.0,58,0.0,0.0,0.0,...,1.0,1,1.0,1,0.0,0.0,0.0,0.0,1.0,1.0
3,06.01.2013,0,25,2554,1709.05,1.0,58,0.0,0.0,0.0,...,0.0,0,0.0,2,0.0,0.0,1.0,1.0,0.0,0.0
4,15.01.2013,0,25,2555,1099.00,1.0,56,0.0,0.0,0.0,...,0.0,0,0.0,0,0.0,0.0,0.0,0.0,0.0,0.0


In [27]:
train_df['item_cumavg'] = train_df.item_cumsum / train_df.item_cumcount
train_df['item_category_cumavg'] = train_df.item_category_cumsum / train_df.item_category_cumcount

train_df.item_cumavg.fillna(0, inplace=True)
train_df.item_category_cumavg.fillna(0, inplace=True)

In [32]:
test_df['item_cumavg'] = test_df.item_cumsum / test_df.item_cumcount
test_df['item_category_cumavg'] = test_df.item_category_cumsum / test_df.item_category_cumcount

test_df.item_cumavg.fillna(0, inplace=True)
test_df.item_category_cumavg.fillna(0, inplace=True)

In [33]:
test_df.columns

Index(['shop_id', 'item_id', 'item_category_id', 'date_block_num',
       'item_lag_1', 'item_lag_2', 'item_lag_3', 'item_lag_4', 'item_lag_5',
       'item_lag_6', 'item_category_lag_1', 'item_category_lag_2',
       'item_category_lag_3', 'item_category_lag_4', 'item_category_lag_5',
       'item_category_lag_6', 'item_cumsum', 'item_cumcount',
       'item_category_cumsum', 'item_category_cumcount', 'item_price',
       'item_cumavg', 'item_category_cumavg'],
      dtype='object')

In [34]:
columns = ['shop_id', 'item_id', 'item_category_id', 'date_block_num',
       'item_lag_1', 'item_lag_2', 'item_lag_3', 'item_lag_4', 'item_lag_5',
       'item_lag_6', 'item_category_lag_1', 'item_category_lag_2',
       'item_category_lag_3', 'item_category_lag_4', 'item_category_lag_5',
       'item_category_lag_6', 'item_price',
       'item_cumavg', 'item_category_cumavg']

In [33]:
X = train_df[train_df.item_cnt_day.between(0, 10)][columns].values
y = train_df[train_df.item_cnt_day.between(0, 10)].item_cnt_day.values

X_train, X_valid, y_train, y_valid = train_test_split(X, y, shuffle=False, test_size=0.2)

In [36]:
params = {
    'boosting_type': 'gbdt',
    'objective': 'rmse',
    'verbose': 10,
    'num_iterations': 250
}

lgb_train = lgb.Dataset(X_train, y_train)

lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train)

gbm = lgb.train(
    params,
    lgb_train,
    valid_sets=lgb_eval,
    early_stopping_rounds=30,
    verbose_eval=10,
    categorical_feature=category_columns)

C:\Users\kipil\Anaconda3\lib\site-packages\lightgbm\engine.py:116: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))
C:\Users\kipil\Anaconda3\lib\site-packages\lightgbm\basic.py:1190: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is [0, 1, 2]
  'New categorical_feature is {}'.format(sorted(list(categorical_feature))))
C:\Users\kipil\Anaconda3\lib\site-packages\lightgbm\basic.py:752: UserWarning: categorical_feature in param dict is overridden.
  warnings.warn('categorical_feature in param dict is overridden.')


Training until validation scores don't improve for 30 rounds.
[10]	valid_0's rmse: 0.797399
[20]	valid_0's rmse: 0.786809
[30]	valid_0's rmse: 0.791417
[40]	valid_0's rmse: 0.792717
Early stopping, best iteration is:
[19]	valid_0's rmse: 0.786524


In [37]:
X_test = test_df[columns].values
y_pred = gbm.predict(X_test, num_iteration=gbm.best_iteration)

In [38]:
submissions_df.item_cnt_month = y_pred
submissions_df.to_csv('submissions/lagged_features_submission_2.csv', index=False)